In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pywsd
!pip install -U wn==0.0.23

     |████████████████████████████████| 26.8MB 115kB/s 
     |████████████████████████████████| 51kB 7.0MB/s 
     |████████████████████████████████| 61kB 6.4MB/s 
  Created wheel for pywsd: filename=pywsd-1.2.4-cp36-none-any.whl size=26940454 sha256=25a5540a13d950d6d0f7d1c2036f73c5913874b09c3f8f88c872a89581c0de8f
  Stored in directory: /root/.cache/pip/wheels/71/4d/d2/405b948047f7f3851f16ab9d893ce7c1a3010182900884536b
Successfully built pywsd
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.25.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
     |████████████████████████████████| 31.6MB 109kB/s 
  Created wheel for wn: filename=wn-0.0.23-cp36-none-any.whl size=31792944 sha256=eb79206d9719b93a28cd1c6fd5a993477f8da2a16a87f908102fb5c1fcea1f

In [2]:
%cd /content/drive/MyDrive/task

/content/drive/MyDrive/task


In [3]:
import os

import numpy as np
import nltk

from scipy import sparse
from tensorflow import keras

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

from Datasets import SimilarityDataset
from models import SimilarityModel
from utility import top_k_metric, calculating_class_weights, get_weighted_loss

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [27]:
import json
from tensorflow.keras.preprocessing.text import tokenizer_from_json

if os.path.exists("/content/drive/MyDrive/input_tokenizer.json"):
    with open("/content/drive/MyDrive/input_tokenizer.json") as f:
        data = json.load(f)
        input_tokenizer = tokenizer_from_json(data)
else:
    input_tokenizer = None

if os.path.exists("/content/drive/MyDrive/target_tokenizer.json"):
    with open("/content/drive/MyDrive/target_tokenizer.json") as f:
        data = json.load(f)
        target_tokenizer = tokenizer_from_json(data)
else:
    target_tokenizer = None

if os.path.exists("/content/drive/MyDrive/new_vocab_sim_mat.npz"):
    sim_mat = sparse.load_npz("/content/drive/MyDrive/new_vocab_sim_mat.npz")
else:
    sim_mat = None

In [28]:
train_ds = SimilarityDataset(mode="train", 
                             tag_func=nltk.pos_tag_sents, 
                             input_tokenizer=input_tokenizer, 
                             target_tokenizer=target_tokenizer, 
                             sim_mat=sim_mat)
val_ds = SimilarityDataset(mode="valid", 
                           tag_func=nltk.pos_tag_sents, 
                           input_tokenizer=train_ds.input_tokenizer, 
                           target_tokenizer=train_ds.target_tokenizer)
test_ds = SimilarityDataset(mode="test", 
                            tag_func=nltk.pos_tag_sents, 
                            input_tokenizer=train_ds.input_tokenizer, 
                            target_tokenizer=train_ds.target_tokenizer)

In [29]:
train_data, train_labels = train_ds.get_data_target()
val_data, val_labels = val_ds.get_data_target()
test_data, test_labels = test_ds.get_data_target()

finished loading corpus
finished loading descriptions
finished extracting contexts
finished extracting targets (OOCs)
finished loading corpus
finished loading descriptions
finished extracting contexts
finished extracting targets (OOCs)
finished loading corpus
finished loading descriptions
finished extracting contexts
finished extracting targets (OOCs)


## Save

In [ ]:
import json

input_tokenizer_json = train_ds.input_tokenizer.to_json()
with open("/content/drive/MyDrive/input_tokenizer.json", "w", encoding="utf-8") as f:
    f.write(json.dumps(input_tokenizer_json, ensure_ascii=False))

target_tokenizer_json = train_ds.target_tokenizer.to_json()
with open("/content/drive/MyDrive/target_tokenizer.json", "w", encoding="utf-8") as f:
    f.write(json.dumps(target_tokenizer_json, ensure_ascii=False))

# Load as:
# with open('tokenizer.json') as f:
#     data = json.load(f)
#     tokenizer = tokenizer_from_json(data)

sparse.save_npz("/content/drive/MyDrive/new_vocab_sim_mat", train_ds.sim_mat)

# Load as:
# sim_mat = sparse.load_npz("/content/drive/MyDrive/sim_mat")

## Model

In [31]:
class_weights = calculating_class_weights(train_labels)

In [35]:
model = SimilarityModel()
model.build_model(
        len(train_ds.input_tokenizer.word_index) + 1,
        train_labels.shape[-1],
        (train_data.shape[1],),
        train_ds.sim_mat.toarray(),
        loss=get_weighted_loss(class_weights),
        metrics=[
            keras.metrics.Precision(name='precision', top_k=5),
            keras.metrics.Recall(name='recall', top_k=5),
        ],
        hidden_sizes=[1024, 1024])

In [36]:
model.train(train_data, train_labels, val_data, val_labels, batch_size=32, epochs=50)

Epoch 1/50
270/270 [==============================] - 13s 45ms/step - loss: 0.8895 - precision: 2.8692e-04 - recall: 6.5784e-04 - val_loss: 0.6283 - val_precision: 0.0011 - val_recall: 0.0027
Epoch 2/50
270/270 [==============================] - 12s 44ms/step - loss: 0.7246 - precision: 0.0010 - recall: 0.0024 - val_loss: 0.6205 - val_precision: 0.0017 - val_recall: 0.0041
Epoch 3/50
270/270 [==============================] - 12s 44ms/step - loss: 0.7015 - precision: 0.0017 - recall: 0.0040 - val_loss: 0.6153 - val_precision: 0.0013 - val_recall: 0.0032
Epoch 4/50
270/270 [==============================] - 12s 44ms/step - loss: 0.6879 - precision: 0.0026 - recall: 0.0059 - val_loss: 0.6147 - val_precision: 0.0026 - val_recall: 0.0064
Epoch 5/50
270/270 [==============================] - 12s 44ms/step - loss: 0.6859 - precision: 0.0029 - recall: 0.0066 - val_loss: 0.6136 - val_precision: 0.0024 - val_recall: 0.0059
Epoch 6/50
270/270 [==============================] - 12s 44ms/step - lo

## Predictions

In [37]:
train_pred = model.infer(train_data)
val_pred = model.infer(val_data)
test_pred = model.infer(test_data)

In [38]:
for i in range(1, 11):
    metric = top_k_metric(train_pred, train_labels, i)
    print(f"top {i}: \nprecision: {metric[0]}\nrecall: {metric[1]}\nf1: {metric[2]}")

top 1: 
precision: 0.021946121690664188
recall: 0.010412934333045805
f1: 0.014124239215289568
top 2: 
precision: 0.021191360891778914
recall: 0.020380753323159268
f1: 0.020778154158067646
top 3: 
precision: 0.01931413531506425
recall: 0.02731888505518324
f1: 0.022629486077600822
top 4: 
precision: 0.017620761727821645
recall: 0.032897756176320965
f1: 0.02294934796237914
top 5: 
precision: 0.016720854621458434
recall: 0.03926014088868246
f1: 0.023453070179815495
top 6: 
precision: 0.015811271094596686
recall: 0.0446800422444872
f1: 0.02335701512994521
top 7: 
precision: 0.015028863380001325
recall: 0.049570643399022396
f1: 0.023064895212004312
top 8: 
precision: 0.014282396655829076
recall: 0.053354123813947316
f1: 0.022532937952523
top 9: 
precision: 0.013830830365897714
recall: 0.0579233295733528
f1: 0.022329792341943402
top 10: 
precision: 0.013446353924756153
recall: 0.06236508305133479
f1: 0.022122862003513467


In [39]:
for i in range(1, 11):
    metric = top_k_metric(val_pred, val_labels, i)
    print(f"top {i}: \nprecision: {metric[0]}\nrecall: {metric[1]}\nf1: {metric[2]}")

top 1: 
precision: 0.015065913370998116
recall: 0.008866917765222849
f1: 0.011163594825723194
top 2: 
precision: 0.01224105461393597
recall: 0.013135593220338982
f1: 0.01267255746674633
top 3: 
precision: 0.011299435028248588
recall: 0.016650973006905212
f1: 0.013462886653533691
top 4: 
precision: 0.011064030131826741
recall: 0.02170433145009416
f1: 0.014656660605667032
top 5: 
precision: 0.011487758945386064
recall: 0.02715003138731952
f1: 0.016144454082467823
top 6: 
precision: 0.010514752040175768
recall: 0.029613935969868173
f1: 0.015519231208300096
top 7: 
precision: 0.01022329835889158
recall: 0.03427495291902071
f1: 0.015749071474277957
top 8: 
precision: 0.009887005649717515
recall: 0.0374450721908349
f1: 0.015643498329062328
top 9: 
precision: 0.009102322661644695
recall: 0.03833961079723791
f1: 0.014711858592390336
top 10: 
precision: 0.008662900188323918
recall: 0.040725047080979286
f1: 0.014286765801525622


In [40]:
for i in range(1, 11):
    metric = top_k_metric(test_pred, test_labels, i)
    print(f"top {i}: \nprecision: {metric[0]}\nrecall: {metric[1]}\nf1: {metric[2]}")

top 1: 
precision: 0.015065913370998116
recall: 0.008866917765222849
f1: 0.011163594825723194
top 2: 
precision: 0.01224105461393597
recall: 0.014908976773383553
f1: 0.013443932812993262
top 3: 
precision: 0.011299435028248588
recall: 0.019617074701820465
f1: 0.014339384553614958
top 4: 
precision: 0.01012241054613936
recall: 0.02299121155053358
f1: 0.014056238341321478
top 5: 
precision: 0.009792843691148776
recall: 0.026632140615191462
f1: 0.01432008250226581
top 6: 
precision: 0.008631512868801004
recall: 0.027369742623979912
f1: 0.013124113725535267
top 7: 
precision: 0.008205542103847187
recall: 0.030194601381042057
f1: 0.01290427849773633
top 8: 
precision: 0.007650659133709981
recall: 0.03145009416195856
f1: 0.012307381821356453
top 9: 
precision: 0.007428332287089349
recall: 0.03325486503452605
f1: 0.01214399083169011
top 10: 
precision: 0.007062146892655367
recall: 0.035451977401129944
f1: 0.011778065581770747
